# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,9.06,57,20,16.98,CL,1730056022
1,1,kapuskasing,49.4169,-82.4331,2.10,43,10,3.04,CA,1730056023
2,2,invercargill,-46.4000,168.3500,7.79,97,100,1.34,NZ,1730056024
3,3,college,64.8569,-147.8028,-12.03,75,100,0.00,US,1730056025
4,4,grytviken,-54.2811,-36.5092,-0.71,70,84,2.43,GS,1730056027


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display
# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", 
)

# Display the map
hvplot.show(humidity_map)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_lats_df = city_data_df[(city_data_df['Lat'] >= -19) & (city_data_df['Lat'] <= 40)]

# Drop any rows with null values
ideal_weather_city_df = cities_narrow_lats_df.dropna()

# Display sample data
ideal_weather_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,saipan,15.1355,145.7010,28.39,89,40,3.60,MP,1730056028
8,8,laguna,38.4210,-121.4238,23.04,67,0,3.09,US,1730056031
11,11,sassandra,4.9500,-6.0833,24.25,98,51,1.80,CI,1730056035
13,13,touros,-5.1989,-35.4608,27.65,67,59,6.33,BR,1730055898
16,16,afaahiti,-17.7500,-149.2833,23.20,74,100,6.98,PF,1730056041
...,...,...,...,...,...,...,...,...,...,...
590,590,mahavelona,-17.6848,49.5087,23.27,91,94,1.65,MG,1730056770
591,591,minas de matahambre,22.5887,-83.9447,26.81,69,100,4.99,CU,1730056771
592,592,kabinda,-6.1333,24.4833,20.98,91,76,1.74,CD,1730056772
593,593,vila franca do campo,37.7167,-25.4333,19.23,99,84,2.92,PT,1730056660


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city_df[['City', 'Country','Lat', 'Lng', 'Humidity']]
hotel_df = hotel_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:, 'Hotel Name'] = ''
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
5,saipan,MP,15.1355,145.7010,89,
8,laguna,US,38.4210,-121.4238,67,
11,sassandra,CI,4.9500,-6.0833,98,
13,touros,BR,-5.1989,-35.4608,67,
16,afaahiti,PF,-17.7500,-149.2833,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address =requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saipan - nearest hotel: Chalan Kanoa Beach Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
sassandra - nearest hotel: Hotel Le Pollet
touros - nearest hotel: Pousada Atlântico
afaahiti - nearest hotel: Omati Lodge
coxen hole - nearest hotel: Elizabeth Inn
kos - nearest hotel: MARITINA
prescott - nearest hotel: Hotel St. Michael
gebeit - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
n'zeto - nearest hotel: Longe LUVUVAMO
rongelap - nearest hotel: No hotel found
valledupar - nearest hotel: Hotel Calderon
zouerate - nearest hotel: فندق تيرس زمور
sabang - nearest hotel: Sabang Hill
remire-montjoly - nearest hotel: Complexe Belova
daru - nearest hotel: Kuki
horta - nearest hotel: Azoris Faial Garden - Resort Hotel
antalaha - nearest hotel: Hôtel Florida
tarfaya - nearest hotel: Hotel Tarfaya
flying fish cove - nearest hotel: Christmas Island Lodge
miahuatlan de porfirio diaz - nearest hotel: No hotel found
lihue - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
5,saipan,MP,15.1355,145.7010,89,Chalan Kanoa Beach Hotel
8,laguna,US,38.4210,-121.4238,67,Holiday Inn Express & Suites
11,sassandra,CI,4.9500,-6.0833,98,Hotel Le Pollet
13,touros,BR,-5.1989,-35.4608,67,Pousada Atlântico
16,afaahiti,PF,-17.7500,-149.2833,74,Omati Lodge
...,...,...,...,...,...,...
590,mahavelona,MG,-17.6848,49.5087,91,No hotel found
591,minas de matahambre,CU,22.5887,-83.9447,69,No hotel found
592,kabinda,CD,-6.1333,24.4833,91,No hotel found
593,vila franca do campo,PT,37.7167,-25.4333,99,Vinha d'Areia Beach Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points( "Lng", "Lat", geo = True, tiles = "OSM", frame_width = 800, frame_height = 600, size = "Lat", scale = 1, color = "City", hover_cols = ["Hotel Name", "Country"] )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)